<a href="https://colab.research.google.com/github/Erfan-Jadidi/MachineLearning/blob/main/CuML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !nvidia-smi

In [8]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
!pip install dask-ml


fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.12 environment at: /usr
Resolved 173 packages in 11.51s
  × Failed to download `libraft-cu12==25.4.0`
  ├─▶ Failed to fetch:
  │   `https://pypi.nvidia.com/libraft-cu12/libraft_cu12-25.4.0-py3-none-manylinux_2_28_x86_64.whl`
  ├─▶ error sending request for url
  │   (https://pypi.nvidia.com/libraft-cu12/libraft_cu12-25.4.0-py3-none-manylinux_2_28_x86_64.whl)
  ├─▶ client error (SendRequest)
  ├─▶ http2 error
  ╰─▶ connection error detected: flow-control protocol violated
  help: `libraft-cu12` (v25.4.0) was included because `pylibraft-cu12`
        (v25.4.0) depends on `libraft-cu12`

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

      

In [9]:
import cudf
cudf.__version__

'25.02.01'

In [10]:
import cuml
cuml.__version__

'25.02.01'

In [14]:
!pip install dask-ml
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.12 environment at: /usr
Resolved 173 packages in 9.71s
  × Failed to download `libcudf-cu12==25.4.0`
  ├─▶ Failed to fetch:
  │   `https://pypi.nvidia.com/libcudf-cu12/libcudf_cu12-25.4.0-py3-none-manylinux_2_28_x86_64.whl`
  ├─▶ error sending request for url
  │   (https://pypi.nvidia.com/libcudf-cu12/libcudf_cu12-25.4.0-py3-none-manylinux_2_28_x86_64.whl)
  ├─▶ client error (SendRequest)
  ├─▶ http2 error
  ╰─▶ connection error detected: flow-control protocol violated
  help: `libcudf-cu12` (v25.4.0) was included because `cudf-cu12` (v25.4.0)
        depends on `libcudf-cu12`

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Plea

In [19]:
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
import pandas as pd
import numpy as np
import cudf
import time
import random

In [20]:
heart_disease = pd.read_csv("https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/data/heart-disease.csv")

# تنظیم seed و تایمر
np.random.seed(42)
random.seed(42)
start_time = time.time()

# تعریف فضای هایپرپارامترها
param_distributions = {
    "n_estimators": [10, 100, 200, 500, 1000, 1200],
    "max_depth": [None, 5, 10, 20, 30],
    "max_features": ["sqrt", "log2", None],
    "min_samples_split": [2, 4, 6, 8],
    "min_samples_leaf": [1, 2, 4, 8]
}

# آماده‌سازی داده‌ها
X = heart_disease.drop("target", axis=1)
y = heart_disease["target"]

# تبدیل به cudf
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = cudf.Series(y)

# تقسیم داده‌ها برای GPU
X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, test_size=0.2)

# جستجوی دستی تصادفی
n_iter = 30
best_score = 0
best_params = None

for i in range(n_iter):
    params = {key: random.choice(val) for key, val in param_distributions.items()}

    # حذف None چون cuml پشتیبانی نمی‌کنه از None برای بعضی پارامترها
    if params["max_depth"] is None:
        del params["max_depth"]
    if params["max_features"] is None:
        del params["max_features"]

    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = accuracy_score(y_test, preds)

    if score > best_score:
        best_score = score
        best_params = params

    print(f"[{i+1}/{n_iter}] Accuracy: {score:.4f}, Params: {params}")

end_time = time.time()

print("\n[RESULT] Best Score:", best_score)
print("[RESULT] Best Params:", best_params)
print(f"[TIME] Total time taken for {n_iter} random searches: {end_time - start_time:.2f} seconds.")

[1/30] Accuracy: 0.8167, Params: {'n_estimators': 1200, 'max_features': 'sqrt', 'min_samples_split': 6, 'min_samples_leaf': 2}
[2/30] Accuracy: 0.7833, Params: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1}
[3/30] Accuracy: 0.8167, Params: {'n_estimators': 1000, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 1}
[4/30] Accuracy: 0.8167, Params: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 2}
[5/30] Accuracy: 0.8333, Params: {'n_estimators': 1200, 'max_depth': 30, 'max_features': 'log2', 'min_samples_split': 4, 'min_samples_leaf': 8}
[6/30] Accuracy: 0.8500, Params: {'n_estimators': 1000, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 4, 'min_samples_leaf': 8}
[7/30] Accuracy: 0.8167, Params: {'n_estimators': 200, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 4, 'min_samples_leaf': 4}
[8/30] Accuracy: 0.8000, Params: {'n_estimators': 10, 'max_fea